# Geocoding and Network Analysis Demo

## What is the Network Analysis?

The ArcGIS Network Analyst allows you to solve common `network` problems

Modern routing algorithm area varients of **Dijkstra's algorithm**

### Examples:

- finding the best route across a city
- finding the closest emergency vehicle or facility, 
- identifying a service area around a location
- servicing a set of orders with a fleet of vehicles 
- choosing the best facilities to open or close.

# Constructing drive time based service areas
This sample shows how the `network` module of the ArcGIS API for Python can be used to construct service areas. In this sample, we generate service areas for two of the fire stations in central Tokyo, Japan. We later observe how the service area varies by time of day for a fire station in the city of Los Angeles.

##### Connect to the GIS

In [ ]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS

my_gis = GIS(profile='agol_profile')

In [ ]:
my_gis.users.me

### Create a Network Layer

In [ ]:
from arcgis.network import ServiceAreaLayer

In [ ]:
service_area_url = my_gis.properties.helperServices.serviceArea.url
service_area_url

In [ ]:
sa_layer = ServiceAreaLayer(service_area_url, gis=my_gis)

## Calculate service area from the Resort

In [ ]:
from arcgis.geocoding import geocode
ps_geocode = geocode('Pointe Hilton Squaw Peak Resort', as_featureset=True)
type(ps_geocode)

# Display the Convetion Center

In [ ]:
map1 = my_gis.map('Pointe Hilton Squaw Peak Resort', zoomlevel=10)
map1

In [ ]:
map1.draw(ps_geocode)

## Compute the service area


In [ ]:
travel_modes = sa_layer.retrieve_travel_modes()

for t in travel_modes['supportedTravelModes']:
    print(t['name'])

In [ ]:
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]

In [ ]:
result = sa_layer.solve_service_area(ps_geocode, default_breaks=[5,10,15], 
                                     travel_direction='esriNATravelDirectionToFacility',
                                     travel_mode=truck_mode)

#### Read the result back as a `FeatureSet`
The `result` variable contains the service area as a dictionary. We inspect its keys and construct `Feature` and `FeatureSet` objects out of it to display in the map

In [ ]:
result.keys()

In [ ]:
result['saPolygons'].keys()

In [ ]:
from arcgis.features import FeatureSet
fs = FeatureSet.from_dict(result['saPolygons'])
from arcgis.features import Feature

In [ ]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)

In [ ]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

### Visualize the service area on the map
From the DataFrame above, we know, there are 3 service area polygons for each fire station. The drive times are given as a range between `FromBreak` and `ToBreak` columns. Let us use this information to visualize the polygons with different colors and appropriate popup messags on the map

In [ ]:
colors = {5: [0, 128, 0, 90], 
          10: [255, 255, 0, 90], 
          15: [255, 0, 0, 90]}

fill_symbol = {"type": "esriSFS","style": "esriSFSSolid",
               "color": [115,76,0,255]}

In [ ]:
for service_area in service_area_fset.features:
    
    #set color based on drive time
    fill_symbol['color'] = colors[service_area.attributes['ToBreak']]
    
    #set popup
    popup={"title": "Service area", 
            "content": "{} minutes".format(service_area.attributes['ToBreak'])}
    
    map1.draw(service_area.geometry, symbol=fill_symbol, popup=popup)

Click the drive time areas to explore their attributes. Because the content of the pop-ups may include HTML source code, it is also possible to have the pop-up windows include other resources such as tables and images.

### Driving directions

In [ ]:
from arcgis.network import RouteLayer

In [ ]:
route_service_url = my_gis.properties.helperServices.route.url
route_service_url

In [ ]:
route_service = RouteLayer(route_service_url, gis=my_gis)
route_service

In [ ]:
stops = [ '7677 N 16th St, Phoenix, AZ 85020', '2501 E Telawa Trail, Phoenix, AZ 85016']
from arcgis.geocoding import geocode, batch_geocode
stops_geocoded = batch_geocode(stops)

#### data wrangling

In [ ]:
stops_geocoded = [item['location'] for item in stops_geocoded]
stops_geocoded

In [ ]:
stops_geocoded2 = '{},{};{},{}'.format(stops_geocoded[0]['x'],stops_geocoded[0]['y'],
                                       stops_geocoded[1]['x'],stops_geocoded[1]['y'])
stops_geocoded2

### Driving directions

In [ ]:
result = route_service.solve(stops_geocoded2, 
                             return_routes=True,
                           return_stops=True, return_directions=True,
                           return_barriers=False, return_polygon_barriers=False,
                           return_polyline_barriers=False)

In [ ]:
ext = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
       'xmin': -12478973.785298374,
       'ymin': 3962066.811862506,
       'xmax': -12460304.041139744,
       'ymax': 3969710.514691013}

In [ ]:
map2 = my_gis.map('Phoenix, AZ', zoomlevel=13)
map2.extent = ext
map2

In [ ]:
from arcgis.features import Feature, FeatureSet
features = result['routes']['features']
routes = FeatureSet(features)
stop_features = result['stops']['features']
stop_fset = FeatureSet(stop_features)

In [ ]:
len(result['stops'])

In [ ]:
stop_features = result['stops']['features']
stop_fset = FeatureSet(stop_features)

In [ ]:
ln_symbol = { "type": "simple-line", "width": 2, "color": [255, 0, 0, 1] }
symbol = {
    "type" : "simple-marker",
    "style" : "square",
    "outline" : {
        "style": "dash-dot"
    },
    "color": [255, 211, 127, 1]
}
map2.draw(routes, symbol=ln_symbol)
map2.draw(stop_fset, symbol=symbol)


# Visualizing the Instructions

In [ ]:
import pandas as pd
text = [att['attributes'] for att in result['directions'][0]['features']]
df = pd.DataFrame(text)[['text', 'length','time']]
df